## ANALISIS DE TENDENCIAS DE CONSUMOS E INFRAESTRUCTURAS

_By JoseForguez & DiegoTondo, 2020-08-09_

In [ ]:
import pandas as pd
import numpy as np
import warnings
import glob
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import plotly.graph_objs as go
import plotly.offline as plotly
from plotly.subplots import make_subplots

import matplotlib.dates as md

In [ ]:
#pd.set_option('display.max_columns', 50)

#### LOAD TRANSACTIONS

In [ ]:
df_transa = pd.read_csv('../dataset/data_csv/sis_transa_201801_202007.csv', parse_dates=['fecha'])

In [ ]:
df_transa.columns

In [ ]:
c2drop = ['acum_usuario', 
          'acum_vehiculo',
          'campovar1',
          'campovar2',
          'km_transaccion',
          'local_price',
          'odometro',
          'tag_description_1',
          'tag_description_2',
          'user_tag_id',
          'valor',
          'veh_efficiency'
         ]

In [ ]:
df_transa.drop(columns=c2drop, axis=1, inplace=True)

In [ ]:
df_transa[:5]

In [ ]:
df_transa[pd.isna(df_transa['cantidad'])]

In [ ]:
df_transa.dropna(subset=['cantidad'], inplace=True)

In [ ]:
len(df_transa[df_transa['cantidad'].astype('string').str.contains(',')])

In [ ]:
df_transa['cantidad'] = df_transa['cantidad'].astype('string').str.replace(',', '.').astype('float64')

In [ ]:
df_transa.dtypes

In [ ]:
df_transa.shape

In [ ]:
df_transa = df_transa[df_transa['cantidad']>=0]
df_transa.shape

In [ ]:
df_transa = df_transa[~(df_transa['fecha']<'2018-01-01')]
df_transa.shape

In [ ]:
df_transa[df_transa['id_equipo'].isna()]

In [ ]:
df_transa[['id_vehiculo', 'cantidad']]

#### LOAD SITES

In [ ]:
df_equipo = pd.read_csv('../dataset/data_csv/fs_equipo.csv')
df_equipo.columns

In [ ]:
df_equipo.drop(axis=1, columns=['direccion_ip', 'pass', 'sync', 'online'], inplace=True)

In [ ]:
df_equipo.shape

#### LOAD COMPANIES

In [ ]:
df_emp = pd.read_csv('../dataset/data_csv/fs_empresa_tagged.csv')

In [ ]:
df_emp.columns

In [ ]:
df_emp.drop(axis=1, columns=['cuit', 'telefono', 'email', 'pais', 'id_pais', 'provincia', 'ciudad', 'direccion', 'cp', 'ultima_fecha_sync_db'], inplace=True)

In [ ]:
df_emp = df_emp.convert_dtypes()
df_emp.dtypes

#### LOAD VEHICULES

In [ ]:
df_veh = pd.read_csv('../dataset/data_csv/fs_vehiculos.csv')

In [ ]:
df_veh.columns

In [ ]:
df_veh

In [ ]:
df_veh.drop(df_veh.loc[:,'id_equipo':'sync'], axis=1, inplace=True)

In [ ]:
df_veh

#### Merge Dataframes

In [ ]:
df_equipo.shape, df_emp.shape

In [ ]:
dfe = pd.merge(df_equipo, df_emp, on='id_empresa')
dfe.shape

In [ ]:
df_transa.shape

In [ ]:
df = pd.merge(df_transa, dfe, on='id_equipo', how='left')
df.shape

In [ ]:
df = df[df['baja']==0]

In [ ]:
df.shape

In [ ]:
dfa = pd.merge(df, df_veh, on='id_vehiculo', how='left')
dfa.shape

In [ ]:
dfa[['fecha', 'id_vehiculo', 'main_id', 'id_empresa', 'cantidad', 'segmento', 'id_tanque', 'id_bomba', 'empresa', 'descripcion']]

In [ ]:
list(dfa[dfa['id_empresa'].isnull()]['id_equipo'].unique())

In [ ]:
dfa = dfa.dropna(axis=0, subset=['id_empresa'])
dfa.shape

In [ ]:
dfa[:5]

In [ ]:
#dfa.to_csv('../dataset/data_csv/sis_transa_201801_202007_merged.csv', index=False)